In [2]:
import os
import cv2
import pycolmap
from hloc.utils import viz_3d
import numpy as np
import pixtrack.utils.pose_utils as pose_utils
from pixloc.utils.colmap import Camera as ColCamera
from pixloc.pixlib.geometry import Camera as PixCamera, Pose
import matplotlib.pyplot as plt
from pixtrack.utils.ingp_utils import load_nerf2sfm, initialize_ingp, sfm_to_nerf_pose, nerf_to_sfm_pose
from pixtrack.visualization.run_vis_on_poses import get_nerf_image
from scipy.spatial.transform import Rotation as R

In [3]:
sfm_path = '/data/aiscience/Sample_data/full_res/ref'
model = pycolmap.Reconstruction(sfm_path)

nerf_path = '/data/aiscience/logs/ingp/sample_data/6111223687/transforms_base.msgpack'
nerf2sfm_path = '/data/aiscience/Sample_data/full_res/nerf2sfm.pkl'
nerf2sfm = load_nerf2sfm(nerf2sfm_path)
testbed = initialize_ingp(nerf_path)

16:13:39 INFO     Loading network config from: /data/aiscience/logs/ingp/sample_data/6111223687/transforms_base.msgpack
16:13:39 INFO     GridEncoding:  Nmin=16 b=1.44727 F=2 T=2^19 L=16
16:13:39 INFO     Density model: 3--[HashGrid]-->32--[FullyFusedMLP(neurons=64,layers=3)]-->1
16:13:39 INFO     Color model:   3--[Composite]-->16+16--[FullyFusedMLP(neurons=64,layers=4)]-->3
16:13:39 INFO       total_encoding_params=12599920 total_network_params=10240


In [4]:
camera = model.cameras[1]
camera = ColCamera(None, 
                camera.model_name,
                int(camera.width),
                int(camera.height),
                camera.params)
camera = PixCamera.from_colmap(camera)
camera = camera.scale(1.0)
#nerf_img = get_nerf_image(self.testbed, nerf_pose, ref_camera)

['6111223687-28.jpg',
 '6111223687-69.jpg',
 '6111223687-53.jpg',
 '6111223687-19.jpg',
 '6111223687-56.jpg',
 '6111223687-31.jpg',
 '6111223687-25.jpg',
 '6111223687-45.jpg',
 '6111223687-3.jpg',
 '6111223687-24.jpg',
 '6111223687-41.jpg',
 '6111223687-60.jpg',
 '6111223687-52.jpg',
 '6111223687-46.jpg',
 '6111223687-68.jpg',
 '6111223687-73.jpg',
 '6111223687-70.jpg',
 '6111223687-39.jpg',
 '6111223687-55.jpg',
 '6111223687-67.jpg',
 '6111223687-7.jpg',
 '6111223687-71.jpg',
 '6111223687-14.jpg',
 '6111223687-9.jpg',
 '6111223687-6.jpg',
 '6111223687-65.jpg',
 '6111223687-22.jpg',
 '6111223687-12.jpg',
 '6111223687-1.jpg',
 '6111223687-76.jpg',
 '6111223687-34.jpg',
 '6111223687-27.jpg',
 '6111223687-72.jpg',
 '6111223687-58.jpg',
 '6111223687-62.jpg',
 '6111223687-4.jpg',
 '6111223687-20.jpg',
 '6111223687-10.jpg',
 '6111223687-21.jpg',
 '6111223687-5.jpg',
 '6111223687-26.jpg',
 '6111223687-15.jpg',
 '6111223687-47.jpg',
 '6111223687-64.jpg',
 '6111223687-37.jpg',
 '6111223687-8.jp

In [8]:
for image_name in os.listdir('/data/aiscience/Sample_data/small_dilation/mapping/'):
    all_imgs_in_altitude.append(image_name)
    
render_poses = []
for image_name in all_imgs_in_altitude:
    image = model.find_image_with_name(image_name)
    cIw = pose_utils.get_camera_in_world_from_colmap_image(image)
    render_poses.append(sfm_to_nerf_pose(nerf2sfm, cIw))

AttributeError: 'NoneType' object has no attribute 'rotation_matrix'

In [3]:
!pip3 install mediapy

In [4]:
from scipy.spatial.transform import Rotation as R
from scipy.spatial.transform import Slerp
from tqdm.notebook import tqdm

In [9]:
image_renders = []
intermediate_frames = 20
for pose_number in tqdm(range(len(render_poses))):
    pose1 = render_poses[pose_number]
    if pose_number+1<len(render_poses):
        pose2 = render_poses[pose_number+1]
        rotations = np.zeros((2, 3, 3))
        rotations[0, :, :] = pose1[:3, :3]
        rotations[1, :, :] = pose2[:3, :3]
        translations_x = np.linspace(pose1[0, -1], pose2[0, -1], intermediate_frames)
        translations_y = np.linspace(pose1[1, -1], pose2[1, -1], intermediate_frames)
        translations_z = np.linspace(pose1[2, -1], pose2[2, -1], intermediate_frames)
    else:
        break

    interpolation_times = np.arange(0, intermediate_frames).tolist()
    key_times = [0, intermediate_frames-1]    
    key_rotations = R.from_matrix(rotations)
    slerp = Slerp(key_times, key_rotations)
    intermediate_rotations = slerp(interpolation_times).as_matrix()
    #print(translations_x.shape[0])
    for pose_num in range(translations_x.shape[0]):
        camera_pose = np.eye(4)
        camera_pose[0, -1] = translations_x[pose_num]
        camera_pose[1, -1] = translations_y[pose_num]
        camera_pose[2, -1] = translations_z[pose_num]
        camera_pose[:3, :3] = intermediate_rotations[pose_num, :, :]
        nerf_img = get_nerf_image(testbed,camera_pose, camera)
        image_renders.append(nerf_img)


  0%|          | 0/40 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [5]:
import mediapy
from tqdm.notebook import tqdm

In [6]:
mediapy.show_video(image_renders, width=320, height=240, fps=10)

NameError: name 'image_renders' is not defined

In [8]:
import numpy as np
def trans_t(t):
    return np.array(
        [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, t], [0, 0, 0, 1],], dtype=np.float32,
    )


def rot_phi(phi):
    return np.array(
        [
            [1, 0, 0, 0],
            [0, np.cos(phi), -np.sin(phi), 0],
            [0, np.sin(phi), np.cos(phi), 0],
            [0, 0, 0, 1],
        ],
        dtype=np.float32,
    )


def rot_theta(th):
    return np.array(
        [
            [np.cos(th), 0, -np.sin(th), 0],
            [0, 1, 0, 0],
            [np.sin(th), 0, np.cos(th), 0],
            [0, 0, 0, 1],
        ],
        dtype=np.float32,
    )

def pose_spherical(theta, phi, radius, up_axis=0):
    """
    Spherical rendering poses, from NeRF
    """
    c2w = trans_t(radius)
    c2w = rot_phi(phi / 180.0 * np.pi) @ c2w
    c2w = rot_theta(theta / 180.0 * np.pi) @ c2w
    c2w = (
        np.array(
            [[-1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]],
            dtype=np.float32,
        )
        @ c2w
    )
    if up_axis != 0:
        vec_up = np.zeros(3, np.float32)
        up_dim = 2 - up_axis // 2
        other_dim = 1 if up_dim == 0 else 0

        vec_up[up_dim] = -1 if up_axis % 2 else 1
        vec_1 = np.zeros(3, np.float32)
        vec_1[other_dim] = 1
        vec_2 = np.cross(vec_up, vec_1)

        trans = np.eye(4, 4, dtype=np.float32)
        trans[:3, 0] = vec_1
        trans[:3, 1] = vec_2
        trans[:3, 2] = vec_up
        c2w = trans @ c2w
    return c2w

In [9]:
render_output_path = "/data/aiscience/logs/ingp/fourth_render"
if not os.path.exists(render_output_path):
    os.mkdir(render_output_path)

In [12]:
from PIL import Image
import pyngp as ngp


In [23]:
elevation = -4
radius = 3.4
up_axis = 1
num_views = 720
render_poses = np.stack(
        [
            pose_spherical(angle, elevation, radius, up_axis - 1)
            for angle in np.linspace(-180, 180, num_views + 1)[:-1]
        ],
        0,
    )  # (NV, 4, 4)

#for i in range(render_poses.shape[0]):
#    render_poses[i, :3, 3] -= [0.0, 0., -0.5]    
image_renders = []
second_renders = []
#testbed.background_color = [1.0, 1., 0.8, 1.0]
#testbed.exposure = 0.1
testbed.color_space = ngp.ColorSpace.SRGB
for image_num, camera_pose in enumerate(tqdm(render_poses)):
    nerf_img = get_nerf_image(testbed, camera_pose, camera)
    image_renders.append(nerf_img.copy())
    Image.fromarray(nerf_img).save(os.path.join(render_output_path, f"{image_num}.jpg"))
    gray = cv2.cvtColor(nerf_img, cv2.COLOR_RGB2GRAY) # convert to grayscale
    blur = cv2.blur(gray, (3, 3)) # blur the image
    ret, thresh = cv2.threshold(np.uint8(255) - blur, 210, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    img = cv2.fillPoly(nerf_img, pts = contours, color =(0, 255, 75))
    mask = np.array((img != np.array([0, 255, 75]))*np.array([0, 255, 75]), dtype=np.uint8)

    # Plotting the contours images
    result_img = cv2.drawContours(nerf_img.copy(), contours, -1, (0,255,75), 2)
    second_renders.append(result_img)

  0%|          | 0/720 [00:00<?, ?it/s]

In [25]:
mediapy.show_video(image_renders, width=int(model.cameras[1].width/3), height=int(model.cameras[1].height/3), fps=5)

This browser does not support the video tag.


In [26]:
mediapy.write_video('car_2.mp4', image_renders, fps=30)

In [ ]:
camera.shape

In [ ]:
int(model.cameras[1].width)

In [ ]:
from PIL import Image

In [ ]:
for image_num, image in enumerate(image_renders):


In [47]:
testbed.nerf.rendering_min_transmittance = 0.01

In [52]:
ngp.ColorSpace.SRGB

<ColorSpace.SRGB: 1>

In [ ]:
ngp.